In [108]:
%load_ext autoreload
%autoreload 2
%aimport

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Modules to reload:
all-except-skipped

Modules to skip:



In [109]:
from IPython.display import HTML, display
import tabulate

from typing import Set
import numpy as np
import matplotlib.pyplot as plt
import PIL

# Twitter Sentiment Model

[Huggingface](https://huggingface.co/models) offers a variety of pre-trained NLP models to explore. We exemplify in this notebook a [transformer-based twitter sentiment classification model](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment). Before getting started, familiarize yourself with the general Truera API as demonstrated in the [intro notebook using pytorch](intro_demo_pytorch.ipynb).

In [110]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

# Wrap all of the necessary components.
class TwitterSentiment:
    MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"

    model = AutoModelForSequenceClassification.from_pretrained(MODEL)
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    labels = ['negative', 'neutral', 'positive']

    NEGATIVE = labels.index('negative')
    NEUTRAL = labels.index('neutral')
    POSITIVE = labels.index('positive')

task = TwitterSentiment()

This model quantifies tweets (or really any text you give it) according to its sentiment: positive, negative, or neutral. Lets try it out on some examples.

In [111]:
sentences = ["I'm so happy!", "I'm so sad!", "I cannot tell whether I should be happy or sad!", "meh"]

# Input sentences need to be tokenized first.

inputs = task.tokenizer(sentences, padding=True, return_tensors="pt") # pt refers to pytorch tensor

# The tokenizer gives us vocabulary indexes for each input token (in this case,
# words and some word parts like the "'m" part of "I'm" are tokens).

print(inputs)

# Decode helps inspecting the tokenization produced:

print(task.tokenizer.batch_decode(torch.flatten(inputs['input_ids'])))
# Normally decode would give us a single string for each sentence but we would
# not be able to see some of the non-word tokens there. Flattening first gives
# us a string for each input_id.

{'input_ids': tensor([[   0,  100,  437,   98, 1372,  328,    2,    1,    1,    1,    1,    1,
            1],
        [   0,  100,  437,   98, 5074,  328,    2,    1,    1,    1,    1,    1,
            1],
        [   0,  100, 1395, 1137,  549,   38,  197,   28, 1372,   50, 5074,  328,
            2],
        [   0, 1794,  298,    2,    1,    1,    1,    1,    1,    1,    1,    1,
            1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
['<s>', 'I', "'m", ' so', ' happy', '!', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<s>', 'I', "'m", ' so', ' sad', '!', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<s>', 'I', ' cannot', ' tell', ' whether', ' I', ' should', ' be', ' happy', ' or', ' sad', '!', '</s>', '<s>', 'me', 'h', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 

Evaluating huggingface models is straight-forward if we use the structure produced by the tokenizer.

In [112]:
outputs = task.model(**inputs)

print(outputs)

# From logits we can extract the most likely class for each sentence and its readable label.

predictions = [task.labels[i] for i in outputs.logits.argmax(axis=1)]

for sentence, logits, prediction in zip(sentences, outputs.logits, predictions):
    print(logits.detach().numpy(), prediction, sentence)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.3217, -0.8764,  4.0706],
        [ 2.5737, -0.4016, -2.1465],
        [ 0.5973,  0.3778, -0.7691],
        [-0.2267,  0.6011, -0.2009]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
[-2.3217378 -0.8763628  4.0705967] positive I'm so happy!
[ 2.5736785  -0.40158105 -2.1465425 ] negative I'm so sad!
[ 0.59733623  0.37779403 -0.7691068 ] negative I cannot tell whether I should be happy or sad!
[-0.22674142  0.601109   -0.20089385] neutral meh


# Model Wrapper

As in the prior notebooks, we need to wrap the pytorch model with the appropriate Trulens functionality. Here we specify the maximum input size (in terms of tokens) each tweet may have.

In [113]:
from trulens.nn.models import get_model_wrapper
from trulens.nn.quantities import LambdaQoI, ClassQoI
from trulens.nn.attribution import InputAttribution, InternalInfluence
from trulens.nn.attribution import IntegratedGradients
from trulens.nn.attribution import Cut, InputCut, OutputCut
from trulens.nn.distributions import LinearDoi, PointDoi
from trulens.utils.typing import ModelInputs

task.wrapper = get_model_wrapper(task.model, input_shape=(None, task.tokenizer.model_max_length))

INFO: Detected pytorch backend for <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'>.
INFO: Using backend Backend.PYTORCH.
INFO: If this seems incorrect, you can force the correct backend by passing the `backend` parameter directly into your get_model_wrapper call.
DEBUG: Input dtype was not passed in. Defaulting to `torch.float32`.


# Attributions

Applying integrated gradents to the sentiment model is similar as in the prior notebooks except special considerations need to be made for the cuts used as the targets of the attribution (i.e. what do we want to assign importance to). As you may have noted above, the model takes as input integer indexes associated with tokens. As we cannot take gradient with respect to these, we use an alternative: the embedding representation of those same inputs. To instantiate trulens with this regard, we need to find inspect the layer names inside our model:

In [114]:
task.wrapper.print_layer_names()

'roberta_embeddings_word_embeddings':	Embedding(50265, 768, padding_idx=1)
'roberta_embeddings_position_embeddings':	Embedding(514, 768, padding_idx=1)
'roberta_embeddings_token_type_embeddings':	Embedding(1, 768)
'roberta_embeddings_LayerNorm':	LayerNorm((768,), eps=1e-05, elementwise_affine=True)
'roberta_embeddings_dropout':	Dropout(p=0.1, inplace=False)
'roberta_encoder_layer_0_attention_self_query':	Linear(in_features=768, out_features=768, bias=True)
'roberta_encoder_layer_0_attention_self_key':	Linear(in_features=768, out_features=768, bias=True)
'roberta_encoder_layer_0_attention_self_value':	Linear(in_features=768, out_features=768, bias=True)
'roberta_encoder_layer_0_attention_self_dropout':	Dropout(p=0.1, inplace=False)
'roberta_encoder_layer_0_attention_output_dense':	Linear(in_features=768, out_features=768, bias=True)
'roberta_encoder_layer_0_attention_output_LayerNorm':	LayerNorm((768,), eps=1e-05, elementwise_affine=True)
'roberta_encoder_layer_0_attention_output_dropou

## Parameters

Above, `roberta_embeddings_word_embeddings` is the layer that produces a continuous representation of each input token so we will use that layer as the one defining the **distribution of interest**. While most neural NLP models contain a token embedding, the layer name will differ.

The second thing to note is the form of model outputs. Specifically, outputs are structures which contain a 'logits' attribute that stores the model scores.

Putting these things together, we instantiate `IntegratedGradients` to attribute each embedding dimension to the maximum class (i.e. the predicted class).

In [115]:
qoi_cut = OutputCut(accessor=lambda o: o['logits'])

infl_max = IntegratedGradients(
    model = task.wrapper,
    doi_cut=Cut('roberta_embeddings_word_embeddings'),
    qoi_cut=qoi_cut
)

In [118]:
# Alternatively we can look at a particular class:

infl_positive = IntegratedGradients(
    model = task.wrapper,
    doi_cut=Cut('roberta_embeddings_word_embeddings'),
    qoi=ClassQoI(task.POSITIVE),
    qoi_cut=qoi_cut
)

Getting attributions uses the same call as model evaluation.

In [141]:
attrs = infl_max.attributions(**inputs)

for token_ids, token_attr in zip(inputs['input_ids'], attrs):
    for token_id, token_attr in zip(token_ids, token_attr):
        # Not that each `word_attr` has a magnitude for each of the embedding
        # dimensions, of which there are many. We aggregate them for easier
        # interpretation and display.
        attr = token_attr.sum()

        word = task.tokenizer.decode(token_id)

        print(f"{word}({attr})", end=' ')

    print()

<s>(0.01430558506399393) I(0.1570846438407898) 'm(0.2109372615814209)  so(0.16590110957622528)  happy(0.3861175775527954) !(0.033557694405317307) </s>(0.08000269532203674) <pad>(0.0) <pad>(0.0) <pad>(0.0) <pad>(0.0) <pad>(0.0) <pad>(0.0) 
<s>(-0.023036574944853783) I(0.14019989967346191) 'm(0.06912591308355331)  so(0.32284340262413025)  sad(-0.09265868365764618) !(-0.1359325796365738) </s>(-0.6459164619445801) <pad>(0.0) <pad>(0.0) <pad>(0.0) <pad>(0.0) <pad>(0.0) <pad>(0.0) 
<s>(-0.033364128321409225) I(-0.028419066220521927)  cannot(-0.4824056029319763)  tell(-0.03216502070426941)  whether(0.11183696985244751)  I(-0.06631225347518921)  should(-0.02624373510479927)  be(-0.118565633893013)  happy(-0.13628575205802917)  or(-0.09041191637516022)  sad(-0.30004382133483887) !(-0.2053295373916626) </s>(-0.5073875188827515) 
<s>(-0.07814183086156845) me(-0.15589091181755066) h(-0.15215334296226501) </s>(0.05180875211954117) <pad>(0.0) <pad>(0.0) <pad>(0.0) <pad>(0.0) <pad>(0.0) <pad>(0.0) <p

A listing as above is not very readable so Trulens comes with some utilities to present token influences a bit more concisely.

In [142]:
import html

def vis(labels, model, tokenizer, sentences, ig):

    inputs = tokenizer(sentences, padding=True, return_tensors="pt")
    outputs = model(**inputs)

    attrs = ig.attributions(**inputs)

    for i, (sentence_word_id, attr, logits) in enumerate(zip(inputs['input_ids'], attrs, outputs['logits'])):

        logits = logits.detach().numpy()
        pred = logits.argmax()
        pred_name = labels[pred]

        sent = f"{pred_name}: "

        for word_id, attr in zip(sentence_word_id, attr):
            word = tokenizer.decode(word_id)
            mag = attr.sum()
            red = 0.0
            green = 0.0
            if mag > 0:
                green = 1.0 # 0.5 + mag * 0.5
                red = 1.0 - mag * 0.5
            else:
                red = 1.0
                green = 1.0 + mag * 0.5
                #red = 0.5 - mag * 0.5

            blue = min(red, green)
            # blue = 1.0 - max(red, green)

            sent += f"<span style='color: rgb({red*255}, {green*255}, {blue*255});'>{html.escape(word)}</span> "

        display(HTML(sent))

display("MAX")
vis(task.labels, task.model, task.tokenizer, sentences, infl_max)

display("POSITIVE")
vis(task.labels, task.model, task.tokenizer, sentences, infl_positive)

'MAX'

'POSITIVE'

# Baselines

We see in the above results that special tokens such as the sentence end **&lt;/s&gt;** contributes are found to contribute a lot to the model outputs. While this may be useful in some contexts, we are more interested in the contributions of the actual words in these sentences. To focus on the words more, we need to adjust the **baseline** used in the integrated gradients computation. By default in the instantiation so far, the baseline for each token is a zero vector of the same shape as its embedding. By making the basaeline be identicaly to the explained instances on special tokens, we can rid their impact from our measurement. 

In [ ]:
import os

os.environ['TRULENS_BACKEND']="pytorch"
os.environ['CUDA_VISIBLE_DEVICES'] = ""

import sys
import torch
import numpy as np

from trulens.nn.models import get_model_wrapper

from trulens.nn.quantities import LambdaQoI, ClassQoI
from IPython.display import HTML, display
import tabulate

from typing import Set
import numpy as np
import matplotlib.pyplot as plt
import PIL

from trulens.nn.attribution import InputAttribution, InternalInfluence
from trulens.nn.attribution import IntegratedGradients
from trulens.nn.attribution import Cut, InputCut, OutputCut
from trulens.nn.distributions import LinearDoi, PointDoi
from trulens.utils.typing import ModelInputs

def as_tensor(thing):
    if isinstance(thing, torch.Tensor):
        return torch.clone(thing)
    else:
        return torch.Tensor(thing).type_as(thing.dtype).device(thing.device)

%matplotlib inline

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# device = torch.device("cuda:0")
device = torch.device("cpu")

bert = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased").to(device)

bt = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
sentences = ["I'm so happy!", "I'm so sad!"]
inputs = bt(sentences, padding=True, return_tensors="pt")

inputs['input_ids'] = inputs['input_ids'].to(device)
inputs['attention_mask'] = inputs['attention_mask'].to(device)

# outs = bert(**inputs)

model = get_model_wrapper(bert, input_shape=(None, bt.model_max_length))
model.print_layer_names()


In [ ]:
def token_baseline(keep_tokens: Set[int], replacement_token: int, ids_to_embeddings):
    """
    Replace all tokens except those in the keep_tokens set with the specified
    replacement token. Returns a methods
    """

    def base_ids(z=None, model_inputs=None):
        input_ids = model_inputs.kwargs['input_ids']
        ids = (1 - sum(input_ids == v for v in keep_tokens)).bool()

        input_ids[ids] = replacement_token

        return input_ids

    def base_embeddings(z=None, model_inputs=None):
        input_ids = base_ids(z, model_inputs)

        return ids_to_embeddings(input_ids)

    return base_ids, base_embeddings

inputs_baseline_ids, inputs_baseline_embeddings = token_baseline(
    keep_tokens=set([task.tokenizer.cls_token_id, task.tokenizer.sep_token_id]),
    replacement_token=task.tokenizer.pad_token_id,
    ids_to_embeddings=task.model.get_input_embeddings()
)

In [ ]:
sentences = ["I'm so happy!", "I'm so sad!", "I cannot tell whether I should be happy or sad!"]

model_input_tensors = ModelInputs(args=[], kwargs=inputs).map(task.wrapper._to_tensor)

inputs = task.tokenizer(sentences, padding=True, return_tensors="pt")
print(task.tokenizer.batch_decode(inputs['input_ids']))
baseline_word_ids = inputs_baseline_ids(model_inputs=ModelInputs(args=[], kwargs=inputs).map(as_tensor))
baseline_embeddings = inputs_baseline_embeddings(model_inputs=ModelInputs(args=[], kwargs=inputs).map(as_tensor))
print(task.tokenizer.batch_decode(baseline_word_ids))

In [ ]:
type(inputs['input_ids'])

task.wrapper = get_model_wrapper(task.model, input_shape=(None, task.tokenizer.model_max_length))
task.wrapper.print_layer_names()

In [ ]:
infl = IntegratedGradients(
    model = task.wrapper,
    resolution=50,
    baseline = inputs_baseline,
    doi_cut=Cut('roberta_embeddings_word_embeddings'),
    qoi=ClassQoI(task.POSITIVE),
    qoi_cut=OutputCut(accessor=lambda o: o['logits'])
)
attrs_input = infl.attributions(**inputs)

In [ ]:
import os
from pathlib import Path
import random
import pandas as pd
import json

from typing import List

from pathlib import Path
import torch
import torch.nn as nn
from torchtext.data import get_tokenizer

class ToySentiment(nn.Module):
    @staticmethod
    def from_pretrained(model_code_path: Path) -> 'ToySentiment':
        # Not strictly necessary to load or save given fixed parameters can be populated, but 
        # implementing this for better integration testing.
        model = ToySentiment()
        model.load_state_dict(torch.load(model_code_path))

    def to_pretrained(self, model_code_path: Path) -> None:
        torch.save(self.state_dict(), model_code_path)

    def set_parameters(self) -> None:
        """Set model parameters as per fixed specification."""

        Wi = torch.zeros_like(self.lstm.weight_ih_l0) + 0.1
        bi = torch.zeros_like(self.lstm.bias_ih_l0) + 0.1
        Wh = torch.zeros_like(self.lstm.weight_hh_l0) + 0.1
        bh = torch.zeros_like(self.lstm.bias_hh_l0) + 0.1

        big = 20.0 # Multipliers to help dealing with LSTM sigmoids.
        half = 8.0 # Intention here is that sigmoid((x*big) - half) is ~0 if x is ~0; and
                    # ~1 when indicator is >~ 1.

        hs = self.hidden_size

        sneutral = 0
        sgood = 1
        sbad = 2
        sconfused = 3

        wneutral = self.vocab['neutral']
        wgood = self.vocab['good']
        wbad = self.vocab['bad']
        
        # make sure c gate is always big
        bi[0:hs*3] = 100.0
        bh[0:hs*3] = 100.0

        # o gate weights:
        Wi[3*hs,   wneutral] = big # read neutral word
        Wi[3*hs+1, wgood] = big # read good word
        Wi[3*hs+2, wbad] = big # read bad word
        Wh[3*hs,   sneutral] = big # keep prior neutral, good, bad states
        Wh[3*hs+1, sgood] = big # 
        Wh[3*hs+2, sbad] = big #
        bi[3*hs:4*hs] = -half # sigmoid will be 0 unless one of the three words was read

        # set "good to bad" confused if prior was good, and input was bad
        Wh[3*hs+3, sgood] = big    # (prior state was good
        Wi[3*hs+3, wbad] = big    #  and input was bad)
        Wh[3*hs+3, sconfused] = 2*big  # or (was already in this confused state)
        bh[3*hs+3] = -(half*2) # Want at least 2 of first two to fire, or just the last one to fire.

        # set "bad to good" confused if prior was bad, and input was good
        Wh[3*hs+4, sbad] = big     # (prior state was bad
        Wi[3*hs+4, wgood] = big     #  and input was good)
        Wh[3*hs+4, sconfused] = 2*big   # or (was already confused)
        bh[3*hs+4] = -(half*2)  #

        self.lstm.weight_hh_l0 = nn.Parameter(Wh)
        self.lstm.bias_hh_l0 = nn.Parameter(bh)
        self.lstm.weight_ih_l0 = nn.Parameter(Wi)
        self.lstm.bias_ih_l0 = nn.Parameter(bi)

        self.embedding.weight = nn.Parameter(torch.eye(self.emb_size))

        self.lin.weight = nn.Parameter(torch.tensor(
            [
                [10.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 20.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 20.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 30.0, 30.0]
            ]
        ))

    def __init__(self):
        super().__init__()
        
        self.tokenizer = get_tokenizer("basic_english")

        self.vocab = {"[UNKNOWN]": 0, "neutral": 1, "good": 2, "bad": 3}

        self.emb_size = len(self.vocab)

        self.hidden_size = 5
        # 5 states, one for neutral, one for positive, one for negative, and two for confused. Requiring two
        # confused states for simplicity of the model; it is easier to encode semantics of confusion based on 
        # which initial positive/negative state was set first.

        # Identity embedding, each vocab word has its own dimension where its presence is encoded.
        self.embedding = nn.Embedding(
            # padding_idx=0, 
            embedding_dim=self.emb_size, 
            num_embeddings=self.emb_size,
            padding_idx=None,
            max_norm=None,
            norm_type=None
        )

        self.lstm = nn.LSTM(
            input_size=self.emb_size,
            hidden_size=self.hidden_size,
            num_layers=1,
            batch_first=True
        )

        # Linear layer to combine the two types of confused state and weight things so that
        # confused outweighs positive and negative, while positive and negative outweigh neutral 
        # if more than one of these states is set.
        self.lin = torch.nn.Linear(
            in_features=self.hidden_size,
            out_features=4,
            bias=False
        )

        # self.sigmoid = torch.nn.Sigmoid()

        # Finally add a softmax for classification.
        self.softmax = torch.nn.Softmax(dim=1)
        # self.softmax = torch.nn.LogSoftmax(dim=1)

        self.set_parameters()

    def forward(self, word_ids: torch.Tensor, embeds=None) -> torch.Tensor:
        if word_ids is not None:
            batch_size = word_ids.shape[0]
        else:
            batch_size = embeds.shape[0]

        h0 = torch.zeros(1, batch_size, self.hidden_size)
        h0[0, 0, 0] = 1.0 # initial state is neutral
        c0 = torch.zeros(1, batch_size, self.hidden_size)

        if embeds is None:
            embeds = self.embedding(word_ids)
            
        embeds.retain_grad()

        out, (hn, cn) = self.lstm(embeds, (h0, c0))
        hn = hn[0,:,:]

        lin_out = self.lin(hn)

        probits = self.softmax(lin_out)
        # preds = torch.argmax(probits, axis=1)
        # pred_prob = torch.gather(probits, dim=1, index=preds[:,None])
        # pred_prob.backward(torch.ones_like(pred_prob))
        # attr = embeds.grad
        # return preds, probits, attr

        return probits


    def input_of_text(self, texts: List[str]) -> torch.Tensor:
        tokens = [self.tokenizer(text) for text in texts]

        word_ids = [[(self.vocab[t] if t in self.vocab else 0) for t in token] for token in tokens]

        return torch.Tensor(word_ids).int()


def generate_dataset(n, l):
    """Generate random sentiment sentences and their labels."""

    ret = []
    cls = []
    for i in range(n):
        sent = []
        while len(sent) < l:
            r = random.random()

            word = "neutral"

            if r > 0.9 and len(sent) > 0:
                continue
            elif r > 0.8:
                word = "good"
            elif r > 0.7:
                word = "bad"

            sent.append(word)

        ret.append(" ".join(sent))

        gt = 0 # neutral
        if "good" in sent and "bad" in sent:
            gt = 3 # confused
        elif "good" in sent:
            gt = 1 # positive
        elif "bad" in sent:
            gt = 2 # negative

        cls.append(gt)

    return pd.DataFrame(dict(sentence=ret, sentiment=cls))

In [ ]:
pt = ToySentiment()
device = 'cpu'
# Produce a wrapped model from the pytorch model.
model = get_model_wrapper(pt, input_shape=(4), device=device, input_dtype=int, )

# test_data = generate_dataset(5, 4)

test_data = generate_dataset(16, 4)

x_sentences = test_data['sentence']
x_tokens = [pt.tokenizer(s) for s in x_sentences]
x_ids = pt.input_of_text(x_sentences)
labels = test_data['sentiment']

# TODO: define "EmbeddedInfluence" for InputInfluence except with an initial embedding layer and initialize internal influence as below:


cls_names = {0:"neutral", 1:"positive", 2:"negative", 3:"confused"}

infl = {}
attrs = {}
for cls in [0,1,2,3]:
    infl[cls] = InternalInfluence(
        model,
        cuts=(Cut("embedding"), Cut("softmax")),
        #doi=PointDoi(cut=Cut("embedding")),
        doi=LinearDoi(cut=Cut("embedding"), resolution=100),
        qoi=ClassQoI(cls),
        multiply_activation=False
    )
    attrs[cls] = infl[cls].attributions(x_ids)


preds = pt(x_ids)

data = []

for i, (words, label, pred) in enumerate(zip(x_tokens, labels, preds)):
    data.append([f"<b>GT={cls_names[label]} PRED={cls_names[pred.argmax().item()]}</b>"] + [" ".join(words)])

    for cls in [0,1,2,3]:
        # print(f"{cls_names[cls]} | ", end='')

        row = [cls_names[cls]]

        sent = ""

        for word, attr in zip(words, attrs[cls][i]):
            # print(f"{word}[{attr.sum()}] ", end="")
            mag = attr.sum()
            red = 0.0
            green = 0.0
            if mag > 0:
                green = 1.0 # 0.5 + mag * 0.5
                red = 1.0 - mag * 0.5
            else:
                red = 1.0
                green = 1.0 + mag * 0.5
                #red = 0.5 - mag * 0.5

            blue = min(red, green)
            # blue = 1.0 - max(red, green)

            sent += f"<span style='color: rgb({red*255}, {green*255}, {blue*255});'>{word}</span> "

        row.append(sent)
        data.append(row)

        # print()

tab = tabulate.tabulate(data, tablefmt='html')
display(HTML(tab))

In [ ]:
test_data['sentence'][0] = "neutral neutral neutral neutral"
test_data['sentence'][1] = "good neutral neutral neutral"
test_data['sentence'][2] = "neutral good neutral neutral"
test_data['sentence'][3] = "neutral neutral good neutral"
test_data['sentence'][4] = "neutral neutral neutral good"
test_data['sentiment'][0] = 0
test_data['sentiment'][1:] = 1

test_data

In [ ]:
test_data = generate_dataset(4, 2)
pm = ToySentiment()
pm.requires_grad_(True)
pm.train()

word_ids = pm.input_of_text(test_data['sentence'])
# embeds = pm.embedding(word_ids)
# embeds.retain_grad()

probits = pm(word_ids=word_ids)

print(probits)

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

In [ ]:
text = "I'm so sad!"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# text = "Good night 😊"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

In [ ]:
for row in csvreader:
    print(row)

In [ ]:
encoded_input